In [1]:
# import json
# import pandas as pd
# with open('1_input_data/config.json', 'r') as file:
#         config = json.load(file)
# image_opportunity_df = pd.read_csv(config["csv_file_path"]["image_capture_opportunity"])

# image_opportunity_df['SatID'] = image_opportunity_df['SatID'].astype(str)
# image_opportunity_df['OpportunityStartOffset'] = image_opportunity_df['OpportunityStartOffset'].astype(int)
# image_opportunity_df['OpportunityEndOffset'] = image_opportunity_df['OpportunityEndOffset'].astype(int)
# image_opportunity_df_copy = image_opportunity_df.copy()
# image_opportunity_df_copy['X'] = image_opportunity_df_copy[['OpportunityStartTime','OpportunityStartOffset']].apply(lambda a: pd.to_datetime(a['OpportunityStartTime']) - pd.DateOffset(seconds=a['OpportunityStartOffset']),axis=1)
# image_opportunity_df_copy['Y'] = image_opportunity_df_copy[['OpportunityEndTime','OpportunityEndOffset']].apply(lambda a: pd.to_datetime(a['OpportunityEndTime']) - pd.DateOffset(seconds=a['OpportunityEndOffset']),axis=1)
# base_time_stamp = image_opportunity_df_copy["X"].to_list()[0]

# image_opportunity_df['base_time'] = base_time_stamp
# image_opportunity_df['req_date'] = image_opportunity_df[['base_time','OpportunityStartOffset']].apply(lambda a: pd.to_datetime(a['base_time']) + pd.DateOffset(seconds=a['OpportunityStartOffset']),axis=1)#pd.to_datetime(image_opportunity_df['base_time']) + pd.to_timedelta(image_opportunity_df['OpportunityStartOffset'])
# image_opportunity_df[['req_date','OpportunityStartTime','base_time','OpportunityStartOffset']]

# image_opportunity_df['x'] = image_opportunity_df[['Priority','StripID']].apply(lambda a: a['Priority'] if a['Priority']<=0 else a['StripID'],axis=1)
# image_opportunity_df[['Priority','StripID','x']]


In [2]:
#!pip list
import os 
os.getcwd()

import sys
print(sys.path)

sys.path.append("../")
print(sys.path)


['/Users/robin/Documents/Schedule_plan/git_APS_Python_core_develop/APS_Python_core/src/APS_Python_core', '/opt/anaconda3/lib/python312.zip', '/opt/anaconda3/lib/python3.12', '/opt/anaconda3/lib/python3.12/lib-dynload', '', '/opt/anaconda3/lib/python3.12/site-packages', '/opt/anaconda3/lib/python3.12/site-packages/aeosa']
['/Users/robin/Documents/Schedule_plan/git_APS_Python_core_develop/APS_Python_core/src/APS_Python_core', '/opt/anaconda3/lib/python312.zip', '/opt/anaconda3/lib/python3.12', '/opt/anaconda3/lib/python3.12/lib-dynload', '', '/opt/anaconda3/lib/python3.12/site-packages', '/opt/anaconda3/lib/python3.12/site-packages/aeosa', '../']


In [3]:
#image_opportunity_df.columns
result_dict = {'a':2,'b':66,'c':43}
result_dict.update({'d':55,'ww':222})
result_dict

{'a': 2, 'b': 66, 'c': 43, 'd': 55, 'ww': 222}

In [4]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import json
import math

from datetime import datetime as dt
import os
# from APS_Python_core.preprocess_1.preprocess_GSpassSelecion import GSPassPreprocess
# from model_3.MILP_GSpassSelection_v3 import GSpassSelection
# from postprocess_4.postprocess_GSpassSelection import GSpasspostprocess

# from preprocess_1.preprocess_imageAqusuition_test import ImageAquisitionProcess #preprocess_imageAqusuition_test,preprocess_imageAquisition_v3_18112024
# from model_3.MILP_imageCapture_v3_17112024_copy import ImageCapturePlan # MILP_imageCapture_v2_16102024,MILP_imageCapture_v2_25102024 # MILP_imageCapture_v2_07112024 #MILP_imageCapture_v3_17112024
# from postprocess_4.image_capture_postprocess_V3_17112024 import ImagecapturePostProcess # image_capture_postprocess_V3_17112024# image_capture_postprocess_v2_18102024

# from preprocess_1.preprocess_downlink_WIP import DownlinkingPreProcess
# from model_3.MILP_downlink import ImageDownlinkPlan
# from postprocess_4.postprocess_downlink import ImageDownlinkPostProcess

# from result_interpret import interpret_result
# from utils import *

from APS_Python_core.preprocess_1.preprocess_GSpassSelecion import GSPassPreprocess
from APS_Python_core.model_3.MILP_GSpassSelection_v3 import GSpassSelection
from APS_Python_core.postprocess_4.postprocess_GSpassSelection import GSpasspostprocess

from APS_Python_core.preprocess_1.preprocess_imageAqusuition_test import ImageAquisitionProcess #preprocess_imageAqusuition_test,preprocess_imageAquisition_v3_18112024
from APS_Python_core.model_3.MILP_imageCapture_v3_17112024 import ImageCapturePlan # MILP_imageCapture_v3_17112024_copy#MILP_imageCapture_v2_16102024,MILP_imageCapture_v2_25102024 # MILP_imageCapture_v2_07112024 #MILP_imageCapture_v3_17112024# 
from APS_Python_core.postprocess_4.image_capture_postprocess_V3_17112024 import ImagecapturePostProcess # image_capture_postprocess_V3_17112024# image_capture_postprocess_v2_18102024

from APS_Python_core.preprocess_1.preprocess_downlink_WIP import DownlinkingPreProcess
from APS_Python_core.model_3.MILP_downlink import ImageDownlinkPlan
from APS_Python_core.postprocess_4.postprocess_downlink import ImageDownlinkPostProcess

from APS_Python_core.result_interpret import interpret_result
from APS_Python_core.utils import *

# script_dir = os.path.abspath( os.path.dirname( __file__ ) )
# print("script directory: ",script_dir)

def select_gs_pass_oppr(GS_pass_df,config):
    ''' 
    this function selects gs pass based on basic constraints.
    Input->Gs_pass_df['GsID', 'AOS', 'LOS', 'SatID', 'AOSOffset', 'LOSOffset']
           config->{"constraints":{"Thermal_constraints_GS_pass":false}
    Output->df[SatID,gsID,concat_sat_gs_k,start_time,end_time,TW_index,AOSoffset,LOSoffset,duration]

    '''

    obj_preprocess = GSPassPreprocess(GS_pass_df,config)
    data = obj_preprocess.preprocess()
    #print(data['SG1K1G2K2_pair']['domain_of_csgk'])
    #print(max([v[2]  for s in data['satellite_id'] for k,v in data['heatTimeBucket_SCT_dict__s'][s].items() if v[1]<=650]))
    #print([v for s in data['satellite_id'] for k,v in data['heatTimeBucket_SCT_dict__s'][s].items() if s=='FF01'])
    print("optimization_model_starts")
    obj_model = GSpassSelection(data,config)
    result,thermal_profile_gsPass = GSpasspostprocess(obj_model,data,config).get_gsPasses()# 21 seconds

    try :
        result['duration'] = result['end_time'] - result['start_time']
        result = result[result['duration']> 0]
    except:
        print("model is not converged or infeasible or not solved")

    return result
    

def select_img_opprtunity(image_opportunity_df,gs_pass_result_df,eclipse_df_dict,config):
    ''' 
    this function selects strip orpprtunity based on memory power thermal and varous priorities..
    Input->gs_pass_result_df-->df[SatID,gsID,concat_sat_gs_k,start_time,end_time,TW_index,AOSoffset,LOSoffset,duration]
           image_opportunity_df-->df[SatID','StripID','AoiID','OpportunityStartTime','OpportunityEndTime','OpportunityStartOffset','OpportunityEndOffset','OrderValidityEnd','CloudCover','OffNadir']
           config->dict({"constraints":{...},
                    "objective":{"GS_Pass_and_Imaging":true,"total_readout_memory":false},
                    "downlink_schedule_OnlyJustsortImgID":true,
                    "GP_weight":0.4,
                    "DDLP_weight":0.2,
                    "CCLP_weight":0.1,
                    "ONLP_weight":0.3,
                    "min_readout_time_seconds":50})

           eclipse_df_dict-->dict{'sat':df['time_index','SatID','eclipse']}

    Output->img_capture_result--> df[SatID,StripID,AoiID,encoded_strip_id,start_time,end_time,gsID,operation,\
                               camera_memory_value_endofTW,delta_camera_memory_value_in_this_TW,SSD_memory_value_endofTW,\
                                delta_SSD_memory_value_in_this_TW,global_priority,local_priority,mean_global_priority,mean_local_priority]

            data--> dict(['encoded_stripId_list', 'imgery_sat_id_list', 'only_gs_sat_id_list', 'unique_img_opportunities_list', 'TW__csjk', \
                          'csjkList__j', 'csjkSet__s', 'cs1j2k2Domainlist__cs1j1k1', 'camera_memory_capacity__s', 'initial_camera_memory_value__s',\
                          'readout_memory_capacity__s', 'power_capacity__s', 'initial_power_value__s', 'initial_readout_camera_memory_value__s', \
                        'imaging_rate', 'Readout_rate', 'heatCameraTimeBucket_SCT_dict__s', 'heatTimeBucket_SCT_dict__s',\
                        'max_camera_heat_dict', 'max_readout_heat_dict', 'stripid__encodedstripID', 'AOIid__encodedstripID', \
                        'assured_tasking_basedOnDueDateEmergency_list', 'assured_tasking_based_on_input_list', \
                        'GlobalPriority__csjk', 'TotalPriority__csjk', 'Local_Priority__csjk', 'TotalPriority__csj', \
                        'GlobalPriority__csj', 'Local_Priority__csj', 'active_assured_strip_id_list', \
                        'PowerglobalTWindexSortedList__s', 'MemoryglobalTWindexSortedList__s', 'Power_GS_TW_list',\
                        'Memory_onlyGsTW_list', 'Power_image_TW_list', 'Memory_onlyImgTW_list', 'Power_NoimageGs_TW_list', \
                        'Memory_NoimageGs_TW_list', 'prev_tWList__s_TWI_dict__s', 'prev_ImagingTWList__s_TWI_dict__s', \
                        'prev_power_tWList__s_TWI_dict__s', 'TW_df_withoutEclipseDivide_df', 'TW_df_withEclipseDivide_df',\
                        'dedicatedReadoutTWlist__concat_sat_memoryTWindex', 'power_based_df', 'success_metric_before', \
                        'TW__csn', 'memory_based_df', 'dedicated_readout_df', 'dedicatedReadoutTWIndex__sat', \
                        'prev_dedicatedReadoutIndex__s_TWI_dict__s', 'DROPriority__concat_sat_memoryTWindex', 'GS_Pass_time_objective']))

    '''

    #basic flters
    #image_opportunity_df = image_opportunity_df[image_opportunity_df['OpportunityEndOffset']<config['scheduled_Hrs']*3600]
    #image_opportunity_df = image_opportunity_df[image_opportunity_df['CloudCoverLimit']>image_opportunity_df['CloudCover']]
    #image_opportunity_df = image_opportunity_df[image_opportunity_df['OffNadirLimit']>image_opportunity_df['OffNadir']]

    obj_preprocess = ImageAquisitionProcess(image_opportunity_df,gs_pass_result_df,eclipse_df_dict,config)
    data = obj_preprocess.preprocess()
    #print(data['cs1j2k2Domainlist__cs1j1k1'])

    #++++++++++++++++++++++++++  STEP 0  +++++++++++++++++++++++++++++++++++++++++++++++
    '''
    hard code some data
    '''
    data['camera_memory_capacity__s'] = {s:v for s,v in data['camera_memory_capacity__s'].items() }
    data['readout_memory_capacity__s'] = {s:v for s,v in data['readout_memory_capacity__s'].items() }
    data['power_capacity__s']  = {s:720000000 for s,v in data['power_capacity__s'].items() }
    data['initial_power_value__s']  = {s:v*0.3 for s,v in data['power_capacity__s'].items() }
    #++++++++++++++++++++++++++  STEP 1  +++++++++++++++++++++++++++++++++++++++++++++++
    config['objective']['GS_Pass_and_Imaging'] = True
    config['objective']['total_readout_memory'] = False
    obj_model = ImageCapturePlan(data,config)

    #Readout Schedule
    if config['readout_schedule']:
        data['GS_Pass_time_objective'] = obj_model.prob.objective.value()
        config['objective']['GS_Pass_and_Imaging'] = False
        config['objective']['total_readout_memory'] = True
        obj_model = ImageCapturePlan(data,config)

    #++++++++++++++++++++++++++  PostProcess  +++++++++++++++++++++++++++++++++++++++++++++++
    post_obj = ImagecapturePostProcess(obj_model,data)
    img_capture_result= post_obj.get_schedule()
    #.isnull().sum()
    return img_capture_result,data
    #======================================================================================================================================================================================================

    pass

def get_aps_success_metric(img_capture_result,data):
    after_aps_plan_df = img_capture_result[img_capture_result['operation']=='Imaging']
    criteria_list = ['total_opprtunities_ratio','GP_ratio','LP_ratio','conflictImg_gsPass']

    total_conflict_images_list= [data['success_metric_before']['conflict_images']]
    GP_before = data['success_metric_before']['original_Total_GP']
    LP_before = data['success_metric_before']['original_Total_LP']
    TOppr_before = data['success_metric_before']['total_opportunities'] 
    before_list = [TOppr_before,GP_before,LP_before,total_conflict_images_list]

    GP_after= after_aps_plan_df['mean_global_priority'].sum()
    LP_after = after_aps_plan_df['mean_local_priority'].sum()
    TOppr_after= after_aps_plan_df['encoded_strip_id'].nunique()
    fraction_conflict_images_list = [list(after_aps_plan_df[after_aps_plan_df['encoded_strip_id'].isin(total_conflict_images_list)]['encoded_strip_id'].unique())]
    after_list = [TOppr_after,GP_after,LP_after,fraction_conflict_images_list]

    APS_success_metric_df = pd.DataFrame({'criteria':criteria_list,'potential_input':before_list,'APS_selected':after_list})
    #APS_success_metric_df['percentage'] = APS_success_metric_df['APS_selected'] / APS_success_metric_df['potential_input']
    APS_success_metric_df1 = APS_success_metric_df[:-1]
    APS_success_metric_df1['percentage'] = APS_success_metric_df1['APS_selected'] / APS_success_metric_df1['potential_input'] * 100
    APS_success_metric_df2 = APS_success_metric_df[-1:]

    APS_success_metric_df = pd.concat([APS_success_metric_df1,APS_success_metric_df2])

    return APS_success_metric_df
    

def get_downlink_schedule(image_downlink_df,img_capture_result,config):

    downlink_operation_list  = ['downlinking_from_camera','downlinking_from_Readout']
    img_capture_result_downlink = img_capture_result[img_capture_result['operation'].isin(downlink_operation_list)]
    DownlinkingPreProcessObj = DownlinkingPreProcess(image_downlink_df,img_capture_result_downlink,config)
    data_downlink = DownlinkingPreProcessObj.preprocess()
    
    if config['downlink_schedule_OnlyJustsortImgID']:
        downlink_result = pd.DataFrame(data_downlink['LP_DD_Priority_imgID'].items(),columns=['ImageID','computed_priority']).\
            sort_values(by='computed_priority',ascending=False)
    else:
        obj_downlink_model = ImageDownlinkPlan(data_downlink,config)
        downlink_result = ImageDownlinkPostProcess(obj_downlink_model,data_downlink).get_schedule()
        downlink_result = downlink_result[downlink_result['TileStripNo_downLoad']!=0]

    return downlink_result
    


def get_input_files(config,GS_pass_df,image_opportunity_df,image_downlink_df,eclipse_event_df):
    ''' 
    to read input files and preliminary preprocessing.
    Input-->
            GS_pass_df-->df['GsID', 'AOS', 'LOS', 'Eclipse', 'AOSOffset', 'LOSOffset','SatID']
            image_opportunity_df-->df['SatID', 'OpportunityStartTime',
                'OpportunityEndTime', 'StripID', 'OffNadir', 'SunInView', 'EarthInView',
                'MoonInView','OrderValidityStart', 'OrderValidityEnd', 'AoiID','CloudCoverLimit', 'CloudCover',
                'OffNadirLimit', 'Priority','OpportunityStartOffset',
                'OpportunityEndOffset']
            image_downlink_df-->df['ImageID', 'SatID', 'DueDate', 'Priority', 'Tilestrips', 'Sensors',
                                   'Bands', 'EmergencyFlag', 'CaptureDate',delivery_type,assured_downlink_flag]
    Output-->dict('GS_pass_df':df[..],\
                'image_opportunity_df':df[..],\
                'image_downlink_df':df[..],\
                "eclipse_df_dict": {'sat':df ,..},
                "config":{..})
    '''
    # GS PASS
    #GS_pass_df = pd.read_csv(config["csv_file_path"]["gs_pass_opportunity"])#APS_gsPasses_TV1#GS_Passes_mock1#GS_Passes_live1#GS_Passes_new (1)
    
    #GS_pass_df = pd.read_csv("1_input_data/GS_Passes_new (1).csv")
    GS_pass_df_original = GS_pass_df.copy()
    GS_pass_df['SatID'] = GS_pass_df['SatID'].astype(str)
    GS_pass_df['AOSOffset'] = GS_pass_df['AOSOffset'].astype(int)
    GS_pass_df['LOSOffset'] = GS_pass_df['LOSOffset'].astype(int)

    # image Opprtunity
    #image_opportunity_df = pd.read_csv(config["csv_file_path"]["image_capture_opportunity"])#Imaging_mock1#APS_imagingOpportunities_TV1#Imaging_live#Imaging_new (1)
    # change made priority ulta
    #image_opportunity_df['Priority'] = 1/image_opportunity_df['Priority']
    
    image_opportunity_df['SatID'] = image_opportunity_df['SatID'].astype(str)
    image_opportunity_df['OpportunityStartOffset'] = image_opportunity_df['OpportunityStartOffset'].astype(int)
    image_opportunity_df['OpportunityEndOffset'] = image_opportunity_df['OpportunityEndOffset'].astype(int)
    image_opportunity_df_copy = image_opportunity_df.copy()
    image_opportunity_df_copy['X'] = image_opportunity_df_copy[['OpportunityStartTime','OpportunityStartOffset']].apply(lambda a: pd.to_datetime(a['OpportunityStartTime']) - pd.DateOffset(seconds=a['OpportunityStartOffset']),axis=1)
    image_opportunity_df_copy['Y'] = image_opportunity_df_copy[['OpportunityEndTime','OpportunityEndOffset']].apply(lambda a: pd.to_datetime(a['OpportunityEndTime']) - pd.DateOffset(seconds=a['OpportunityEndOffset']),axis=1)
    base_time_stamp = image_opportunity_df_copy["X"].to_list()[0]
    config['base_time_stamp_downlink'] = base_time_stamp

    #image Downlink
    #image_downlink_df = pd.read_csv(config["csv_file_path"]["image_downlink_file"])
    image_downlink_df['assured_downlink_flag'] = [0,0] +[0]*(len(image_downlink_df)-2)
    image_downlink_df['delivery_type'] = 'standard_delivery' # expedited_delivery,super_expedited_delivery
    union_list_of_sat = list(set(image_opportunity_df['SatID']).union(set(GS_pass_df['SatID'])).union(set(image_downlink_df['SatID'])))
    hrs = config['scheduled_Hrs']

    # get dummy eclipse data close to reality
    # satellite_list = eclipse_event_df['SatID'].unique()
    # eclipse_df = pd.DataFrame()
    # for sat in satellite_list:
    #     this_eclipse_df = eclipse_event_df[eclipse_event_df['SatID']==sat]
    #     that_eclipse_df = get_eclipse_data(this_eclipse_df,config)
    #     eclipse_df = pd.concat([that_eclipse_df,eclipse_df])
        
    # min_time_index= min([image_opportunity_df['OpportunityStartOffset'].min(),image_opportunity_df['OpportunityEndOffset'].max(),GS_pass_df['AOSOffset'].min(),GS_pass_df['LOSOffset'].max()])
    # max_time_index= max([image_opportunity_df['OpportunityStartOffset'].min(),image_opportunity_df['OpportunityEndOffset'].max(),GS_pass_df['AOSOffset'].min(),GS_pass_df['LOSOffset'].max()])

    # hrs = (max_time_index - min_time_index)/3600
    # hrs = math.ceil(hrs)
    # while True:
    #     hrs += 1
    #     if hrs % 1.5==0:
    #         break


    # in_orbit_eclipse_event = [1 for i in range(int(1.5*3600*0.4))] + [0 for i in range(int(1.5*3600*0.6))] #
    # eclipse_df  = pd.DataFrame({'time_index': [i for i in range(min_time_index,min_time_index+hrs*3600)] ,"eclipse" : in_orbit_eclipse_event*int(hrs/1.5)})
    # eclipse_df['SatID']= [union_list_of_sat] *len(eclipse_df)
    # eclipse_df = eclipse_df.explode('SatID')
    # eclipse_df_dict = {s: eclipse_df[eclipse_df['SatID']==s] for s in eclipse_df['SatID'].unique()}
    try :
        eclipse_df = pd.DataFrame()
        satellite_list = eclipse_event_df['SatID'].unique()
        for sat in satellite_list:
            this_eclipse_df = eclipse_event_df[eclipse_event_df['SatID']==sat]
            that_eclipse_df = get_eclipse_data(this_eclipse_df,config)
            eclipse_df = pd.concat([that_eclipse_df,eclipse_df])
        eclipse_df_dict = {s: eclipse_df[eclipse_df['SatID']==s] for s in eclipse_df['SatID'].unique()}
    except:
        print("some_error_in_eclipse_data_So_hard_coded_eclipse_data")
        min_time_index= min([image_opportunity_df['OpportunityStartOffset'].min(),image_opportunity_df['OpportunityEndOffset'].max(),GS_pass_df['AOSOffset'].min(),GS_pass_df['LOSOffset'].max()])
        max_time_index= max([image_opportunity_df['OpportunityStartOffset'].min(),image_opportunity_df['OpportunityEndOffset'].max(),GS_pass_df['AOSOffset'].min(),GS_pass_df['LOSOffset'].max()])

        hrs = (max_time_index - min_time_index)/3600
        hrs = math.ceil(hrs)
        while True:
            hrs += 1
            if hrs % 1.5==0:
                break


        in_orbit_eclipse_event = [1 for i in range(int(1.5*3600*0.4))] + [0 for i in range(int(1.5*3600*0.6))] #
        eclipse_df  = pd.DataFrame({'time_index': [i for i in range(min_time_index,min_time_index+hrs*3600)] ,"eclipse" : in_orbit_eclipse_event*int(hrs/1.5)})
        eclipse_df['SatID']= [union_list_of_sat] *len(eclipse_df)
        eclipse_df = eclipse_df.explode('SatID')
        eclipse_df_dict = {s: eclipse_df[eclipse_df['SatID']==s] for s in eclipse_df['SatID'].unique()}


    # get dummy data for assured tasking
    image_opportunity_df['encoded_stripId'] =   image_opportunity_df['StripID'].astype(str)+ '_' + image_opportunity_df['AoiID'].astype(str)
    total_capture_list = list(image_opportunity_df['encoded_stripId'].unique())
    no_of_list = len(total_capture_list)
    assured_capture_df = pd.DataFrame({'encoded_stripId':total_capture_list,'assured_task':[0,0]+[0]*(no_of_list-2)})
    image_opportunity_df = pd.merge(image_opportunity_df,assured_capture_df,on='encoded_stripId',how='left')
    image_opportunity_df = image_opportunity_df.drop(columns=['encoded_stripId'])

    # further processing eclipse data to align with gs pass where entire gs pass is assumed to be in eclipse region
    gsPassInput_df_copy = GS_pass_df_original
    gsPassInput_df_copy['SatID'] = gsPassInput_df_copy['SatID'].astype(str)
    gsPassInput_df_copy['AOSOffset'] = gsPassInput_df_copy['AOSOffset'].astype(int)
    gsPassInput_df_copy['LOSOffset'] = gsPassInput_df_copy['LOSOffset'].astype(int)
    gsPassInput_df_copy['list'] =  gsPassInput_df_copy[['AOSOffset','LOSOffset']].apply(lambda a : [i for i in range(a['AOSOffset'],a['LOSOffset']+1)],axis =1 )

    gsPassInput_df_copy1 = gsPassInput_df_copy[['SatID','list']]
    gsPassInput_df_copy1 = gsPassInput_df_copy1.explode('list')
    gsPassInput_df_grouped_copy1 = gsPassInput_df_copy1.groupby('SatID').agg(time_index_list = ('list',list)).reset_index()
    gsPasstimeIndexList__s = dict(zip(gsPassInput_df_grouped_copy1['SatID'],gsPassInput_df_grouped_copy1['time_index_list']))
    for k,v in eclipse_df_dict.items():
        if k in gsPasstimeIndexList__s.keys():
            this_time_index_list = gsPasstimeIndexList__s[k]
            v.loc[v["time_index"].isin(this_time_index_list), "eclipse"] = 1
            eclipse_df_dict[k] = v

    return {
            'GS_pass_df':GS_pass_df,\
            'image_opportunity_df':image_opportunity_df,\
            'image_downlink_df':image_downlink_df,\
            "eclipse_df_dict": eclipse_df_dict,
            "config":config
            }
    
def get_schedule(config,GS_pass_df,image_opportunity_df,image_downlink_df,eclipse_df):
    ''' 
    to get imaging , readout schedule and stats for schedule.

    Input-->
            GS_pass_df-->df['GsID', 'AOS', 'LOS', 'Eclipse', 'AOSOffset', 'LOSOffset','SatID']
            image_opportunity_df-->df['SatID', 'OpportunityStartTime',
                'OpportunityEndTime', 'StripID', 'OffNadir', 'SunInView', 'EarthInView',
                'MoonInView','OrderValidityStart', 'OrderValidityEnd', 'AoiID','CloudCoverLimit', 'CloudCover',
                'OffNadirLimit', 'Priority','OpportunityStartOffset',
                'OpportunityEndOffset']
            image_downlink_df-->df['ImageID', 'SatID', 'DueDate', 'Priority', 'Tilestrips', 'Sensors',
                                   'Bands', 'EmergencyFlag', 'CaptureDate',delivery_type,assured_downlink_flag]

    Output-->dict{"only_readout_result":df['SatID','start_time','end_time','base_time'],\
        "only_img_capture_result":df['SatID','start_time','end_time','AoiID','StripID','base_time'],\
            "only_gsPass_result":df['SatID','start_time','end_time','gsID','base_time'],\
            "combined_result":df['SatID', 'start_time', 'end_time', 'encoded_strip_id', 'gsID','operation', 'camera_memory_value_endofTW',
                                'delta_camera_memory_value_in_this_TW', 'SSD_memory_value_endofTW','delta_SSD_memory_value_in_this_TW', 'global_priority',
                                'local_priority', 'mean_global_priority', 'mean_local_priority','StripID', 'AoiID', 'base_time']}

            "interpret_extracted_raw_file_df":df['SatID', 'OpportunityStartTime', 'OpportunityEndTime', 'StripID','OffNadir', 'OrderValidityStart', 'OrderValidityEnd', 'AoiID','Priority', 'OpportunityStartOffset', 'OpportunityEndOffset',
                                                'normalized_local_priority_due_date','normalized_local_priority_CC_based','normalized_local_priority_offNadir', 'normalized_GlobalPriority','normalized_Total_Priority', 'camera_memory_value_endofTW',
                                                'delta_camera_memory_value_in_this_TW', 'flag', 'flag_gs_pass_conflict','conflicting_strip_oppr', 'concat_SatID_encodedStripId_TWindex','CloudCoverLimit', 'CloudCover', 'encoded_strip_id', 'base_time']
            "interpret_selected_oppr_conflict_comparision_df"df['conflic_strip_flag_named', 'max_Norm_TP', 'this_flag_norm_TP','max_Norm_GP', 'this_flag_norm_GP', 'max_Norm_LPDD','this_flag_norm_LLDD', 'base_time']
            "interpret_KPI_df: df['criteria', 'before_APS', 'APS_result', 'remarks', 'percentage','base_time']
            
                                                   
    '''
    
    # Open and read the JSON file
    #APS_Python_core/src/APS_Python_core/1_input_data/config.json
    #with open('APS_Python_core/src/APS_Python_core/1_input_data/config.json', 'r') as file:
    #with open('../1_input_data/config.json', 'r') as file:
        #config = json.load(file)
    original_image_opportunity_df = image_opportunity_df.copy()
    # if memory constraint False then thermal_constraint is also False
    #config['constraints']['thermal_constraint_readout'] = config['constraints']['memory_constrant'] and config['constraints']['thermal_constraint_readout']
    #config['constraints']['thermal_constraint_imaging'] = config['constraints']['memory_constrant'] and config['constraints']['thermal_constraint_imaging']

    #======================================================================================================================================================================================================
    # read_input
    input_dict = get_input_files(config,GS_pass_df,image_opportunity_df,image_downlink_df,eclipse_df)
    config = input_dict['config']
    #======================================================================================================================================================================================================
    #gs pass_selection
    gs_pass_result_df = select_gs_pass_oppr(input_dict['GS_pass_df'],config)
    gs_pass_result_df['Eclipse'] = 1 ## dummy
    gs_pass_result_df['duration'] = gs_pass_result_df['end_time'] - gs_pass_result_df['start_time']
    gs_pass_result_df = gs_pass_result_df[gs_pass_result_df['duration']> 0]
    print(gs_pass_result_df['concat_sat_gs_k'].nunique())
    interpret_gs_pass_result_df_copy = gs_pass_result_df.copy()# this not the gsPass result as it is to be get filtered after due to other factors in image capture plan.It is just to get require info in interpret result.

    print("image_capture_plan_starting")
    #======================================================================================================================================================================================================
    #image_selection
    img_capture_result,capture_plan_data_input= select_img_opprtunity(input_dict['image_opportunity_df'],gs_pass_result_df,input_dict['eclipse_df_dict'],config)
    img_capture_result['base_time'] = config['base_time_stamp_downlink']
    #img_capture_result = img_capture_result[img_capture_result['operation']=='Imaging']
    #readout_result = img_capture_result[img_capture_result['operation']=='Readout']
    interpret_img_capture_resul_copy = img_capture_result.copy()
    #======================================================================================================================================================================================================
    # get APS success metrics 
    APS_success_metric_df = get_aps_success_metric(img_capture_result,capture_plan_data_input)
    #======================================================================================================================================================================================================
    print("Downlink_plan_starting")
    try:
        downlink_result = get_downlink_schedule(input_dict['image_downlink_df'],img_capture_result,config)
        downlink_result['base_time'] = config['base_time_stamp_downlink']
    except:
        print("downlink_schedule_error")
    #======================================================================================================================================================================================================
    #img_capture_result[img_capture_result['download_from_']]
    # gs_pass_result_df.to_csv("APS_Python_core/src/APS_Python_core/5_output_data/gs_pass_result_df.csv",index=None)
    # img_capture_result.to_csv("APS_Python_core/src/APS_Python_core/5_output_data/img_capture_schedule.csv",index=None)
    # APS_success_metric_df.to_csv("APS_Python_core/src/APS_Python_core/5_output_data/APS_success_metric.csv",index = None)
    # downlink_result.to_csv("APS_Python_core/src/APS_Python_core/5_output_data/downlink_result.csv",index = None)

    interpret_image_opportunity_df = original_image_opportunity_df
    interpret_result_dict = interpret_result(interpret_image_opportunity_df,interpret_gs_pass_result_df_copy,interpret_img_capture_resul_copy,config)
    for k,v in interpret_result_dict.items():
        v['base_time'] = config['base_time_stamp_downlink']
        #v.to_csv("APS_Python_core/src/APS_Python_core/5_output_data/"+k+".csv",index = None)
    
    only_img_capture_result = img_capture_result[img_capture_result['operation']=='Imaging'][['SatID','start_time','end_time','AoiID','StripID','base_time']]
    only_img_capture_result['start_time'] = only_img_capture_result[['start_time','base_time']].apply(lambda a: pd.to_datetime(a['base_time']) + pd.DateOffset(seconds=a['start_time']),axis=1)
    only_img_capture_result['end_time'] = only_img_capture_result[['end_time','base_time']].apply(lambda a: pd.to_datetime(a['base_time']) + pd.DateOffset(seconds=a['end_time']),axis=1)

    only_readout_result = img_capture_result[img_capture_result['operation']=='Readout'][['SatID','start_time','end_time','base_time']]
    
    if len(only_readout_result):
        only_readout_result['start_time'] = only_readout_result[['start_time','base_time']].apply(lambda a: pd.to_datetime(a['base_time']) + pd.DateOffset(seconds=a['start_time']),axis=1)
        only_readout_result['end_time'] = only_readout_result[['end_time','base_time']].apply(lambda a: pd.to_datetime(a['base_time']) + pd.DateOffset(seconds=a['end_time']),axis=1)

    only_gsPass_result = img_capture_result[img_capture_result['operation']=='downlinking_from_Readout'][['SatID','start_time','end_time','gsID','base_time']]
    only_gsPass_result['start_time'] = only_gsPass_result[['start_time','base_time']].apply(lambda a: pd.to_datetime(a['base_time']) + pd.DateOffset(seconds=a['start_time']),axis=1)
    only_gsPass_result['end_time'] = only_gsPass_result[['end_time','base_time']].apply(lambda a: pd.to_datetime(a['base_time']) + pd.DateOffset(seconds=a['end_time']),axis=1)


    result_dict = {"only_readout_result":only_readout_result,\
                  "only_img_capture_result":only_img_capture_result,\
                  "only_gsPass_result":only_gsPass_result,\
                  "combined_result":img_capture_result}
    result_dict.update(interpret_result_dict)
    #return result_dict
    return result_dict

In [5]:
os.getcwd()

'/Users/robin/Documents/Schedule_plan/git_APS_Python_core_develop/APS_Python_core/src/APS_Python_core'

In [6]:
GS_pass_df = pd.read_csv("1_input_data/GS_Passes_new (1).csv")
image_opportunity_df = pd.read_csv("1_input_data/Imaging_new (1) copy.csv")
image_downlink_df = pd.read_csv("1_input_data/APS_imageTable_TV1.csv")
eclipse_df = pd.DataFrame()
with open('1_input_data/config.json', 'r') as file:
    config = json.load(file)
result_dict = get_schedule(config,GS_pass_df,image_opportunity_df,image_downlink_df,eclipse_df)
for k,v in result_dict.items():
    v.to_csv('5_output_data/'+k+'.csv',index=None)




some_error_in_eclipse_data_So_hard_coded_eclipse_data
=====GA pASS THERMAL======
optimization_model_starts
start_solving
Running HiGHS 1.8.1 (git hash: 4a7f24a): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+09]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 2e+09]
Presolving model
1598487 rows, 57850 cols, 4790687 nonzeros  0s
116721 rows, 57847 cols, 1088253 nonzeros  1s
99263 rows, 41698 cols, 799099 nonzeros  5s
84423 rows, 35391 cols, 678696 nonzeros  6s
71809 rows, 30030 cols, 575763 nonzeros  7s
61087 rows, 25474 cols, 489390 nonzeros  7s
51975 rows, 21601 cols, 412045 nonzeros  8s
44229 rows, 18309 cols, 349726 nonzeros  8s
37645 rows, 15511 cols, 296877 nonzeros  8s
32049 rows, 13132 cols, 251605 nonzeros  8s
27291 rows, 11110 cols, 213337 nonzeros  9s
23247 rows, 9392 cols, 180705 nonzeros  9s
19811 rows, 7931 cols, 150351 nonzeros  9s
16889 rows, 6689 cols, 124451 nonzeros  9s
14405 rows, 5634 cols, 105427 nonzeros

In [7]:
GS_pass_df = pd.read_csv("1_input_data/GS_Passes_new (1).csv")
GS_pass_df['d'] = GS_pass_df['LOSOffset'] - GS_pass_df['AOSOffset']
GS_pass_df.sort_values(by=['SatID','AOSOffset'],ascending=True,inplace=True)
GS_pass_df['los_shift'] = GS_pass_df.groupby('SatID')['LOSOffset'].shift(1)
GS_pass_df['cool_time'] = GS_pass_df['AOSOffset']- GS_pass_df['los_shift']
GS_pass_df.sort_values(by=['SatID','AOSOffset'],ascending=True,inplace=True)
GS_pass_df.groupby("SatID")['cool_time'].min()

SatID
FF01    356.0
FF02    358.0
FF03    344.0
Name: cool_time, dtype: float64

In [8]:
GS_pass_df.columns

Index(['Unnamed: 0', 'GsID', 'AOS', 'LOS', 'Eclipse', 'AOSOffset', 'LOSOffset',
       'SatID', 'd', 'los_shift', 'cool_time'],
      dtype='object')

In [9]:
result_dict['interpret_KPI_df'].columns

Index(['criteria', 'before_APS', 'APS_result', 'remarks', 'percentage',
       'base_time'],
      dtype='object')

In [10]:
result_dict.keys()


dict_keys(['only_readout_result', 'only_img_capture_result', 'only_gsPass_result', 'combined_result', 'interpret_extracted_raw_file_df', 'interpret_selected_oppr_conflict_comparision_df', 'interpret_KPI_df'])

In [11]:
GS_pass_df = GS_pass_df = pd.read_csv("1_input_data/GS_Passes_new (1).csv")
image_opportunity_df = pd.read_csv("1_input_data/Imaging_new (1) copy.csv")
image_downlink_df = pd.read_csv("1_input_data/APS_imageTable_TV1.csv")
image_downlink_df.columns

Index(['ImageID', 'SatID', 'DueDate', 'Priority', 'Tilestrips', 'Sensors',
       'Bands', 'EmergencyFlag', 'CaptureDate'],
      dtype='object')

In [12]:
#-48283.5824309=25 strips 83 gspasses,-48283.5824309=25 strips 83 gspasses,-48283.5824309=26 strips 83 gspasses ,  -48283.5824309= 24 strips 82 gspasses , -48283.5824309 25 strips 82 gs PASss,-48283.5824309 24 strips 83 gs PASss

In [13]:
#GS_pass_df

In [14]:
#max(image_opportunity_df['Priority'].to_list())

In [15]:
result_dict.keys()

dict_keys(['only_readout_result', 'only_img_capture_result', 'only_gsPass_result', 'combined_result', 'interpret_extracted_raw_file_df', 'interpret_selected_oppr_conflict_comparision_df', 'interpret_KPI_df'])

In [16]:
result_dict['only_img_capture_result']['StripID'].nunique(),len(result_dict['only_gsPass_result'])

(26, 83)

In [17]:
result_dict['only_gsPass_result'].head(60)

,SatID,start_time,end_time,gsID,base_time
30,FF01,2024-11-14 12:36:49,2024-11-14 12:44:54,JEJU01,2024-11-13 12:30:44
31,FF01,2024-11-14 12:51:35,2024-11-14 13:02:18,SVALBARD01,2024-11-13 12:30:44
32,FF02,2024-11-14 13:07:41,2024-11-14 13:17:44,JEJU01,2024-11-13 12:30:44
33,FF02,2024-11-14 13:23:42,2024-11-14 13:34:21,SVALBARD01,2024-11-13 12:30:44
34,FF03,2024-11-14 13:39:21,2024-11-14 13:50:02,JEJU01,2024-11-13 12:30:44
35,FF01,2024-11-14 13:51:01,2024-11-14 13:54:32,AWARUA01,2024-11-13 12:30:44
36,FF03,2024-11-14 13:55:46,2024-11-14 14:06:22,SVALBARD01,2024-11-13 12:30:44
37,FF01,2024-11-14 14:11:39,2024-11-14 14:21:49,JEJU01,2024-11-13 12:30:44
38,FF01,2024-11-14 14:27:45,2024-11-14 14:38:18,SVALBARD01,2024-11-13 12:30:44
39,FF02,2024-11-14 14:44:54,2024-11-14 14:53:03,JEJU01,2024-11-13 12:30:44


In [18]:
A = result_dict['combined_result']
A[A['SatID']=='FF02'].sort_values(by='start_time')

,SatID,start_time,end_time,encoded_strip_id,gsID,operation,camera_memory_value_endofTW,delta_camera_memory_value_in_this_TW,SSD_memory_value_endofTW,delta_SSD_memory_value_in_this_TW,global_priority,local_priority,mean_global_priority,mean_local_priority,StripID,AoiID,base_time
32,FF02,88617.0,89220.0,,JEJU01,downlinking_from_Readout,20.0,0.0,NA,NA,no_i_no_g,no_i_no_g,no_i_no_g,no_i_no_g,NaN,NaN,2024-11-13 12:30:44
33,FF02,89578.0,90217.0,,SVALBARD01,downlinking_from_Readout,20.0,0.0,NA,NA,no_i_no_g,no_i_no_g,no_i_no_g,no_i_no_g,NaN,NaN,2024-11-13 12:30:44
5,FF02,91482.0,91498.0,Order 53 - Strip 3_Area 36,,Imaging,132.0,112.0,NA,NA,1000.0,1029.347967,1000.0,1029.347967,Order 53 - Strip 3,Area 36,2024-11-13 12:30:44
1,FF02,91539.0,92164.0,no_i_no_g,no_i_no_g,Readout,0.0,-500.0,850.0,500.0,no_i_no_g,no_i_no_g,no_i_no_g,no_i_no_g,NaN,NaN,2024-11-13 12:30:44
39,FF02,94450.0,94939.0,,JEJU01,downlinking_from_Readout,0.0,0.0,NA,NA,no_i_no_g,no_i_no_g,no_i_no_g,no_i_no_g,NaN,NaN,2024-11-13 12:30:44
40,FF02,95342.0,95974.0,,SVALBARD01,downlinking_from_Readout,0.0,0.0,NA,NA,no_i_no_g,no_i_no_g,no_i_no_g,no_i_no_g,NaN,NaN,2024-11-13 12:30:44
3,FF02,95975.0,96137.5,no_i_no_g,no_i_no_g,Readout,0.0,-130.0,980.0,130.0,no_i_no_g,no_i_no_g,no_i_no_g,no_i_no_g,NaN,NaN,2024-11-13 12:30:44
43,FF02,101098.0,101733.0,,SVALBARD01,downlinking_from_Readout,0.0,0.0,NA,NA,no_i_no_g,no_i_no_g,no_i_no_g,no_i_no_g,NaN,NaN,2024-11-13 12:30:44
7,FF02,102364.0,102380.0,Order 14 - Strip 3_Area 18,,Imaging,112.0,112.0,NA,NA,1000.0,766.940092,1000.0,766.940092,Order 14 - Strip 3,Area 18,2024-11-13 12:30:44
46,FF02,105333.0,105578.0,,MAU01,downlinking_from_Readout,0.0,0.0,NA,NA,no_i_no_g,no_i_no_g,no_i_no_g,no_i_no_g,NaN,NaN,2024-11-13 12:30:44


In [19]:
#capture_plan_data_input['']
#-48283.5824309
#-48271
len(result_dict['interpret_selected_oppr_conflict_comparision_df'])

26

In [20]:
result_dict['interpret_selected_oppr_conflict_comparision_df']#['conflic_strip_flag_named'].nunique()##['interpret_selected_oppr_conflict_comparision_df']#['interpret_extracted_raw_file_df']

,conflic_strip_flag_named,max_Norm_TP,this_flag_norm_TP,max_Norm_GP,this_flag_norm_GP,max_Norm_LPDD,this_flag_norm_LLDD,base_time
0,FF01_Order 11 - Strip 1_Area 13_1.0,403,403,500.0,500.0,1000.000000,1000.000000,2024-11-13 12:30:44
6,FF01_Order 22 - Strip 3_Area 21_1.0,568,568,1000.0,1000.0,752.280794,752.280794,2024-11-13 12:30:44
11,FF01_Order 30 - Strip 0_Area 27_1.0,555,555,1000.0,1000.0,752.280794,752.280794,2024-11-13 12:30:44
16,FF01_Order 47 - Strip 5_Area 35_1.0,506,506,750.0,750.0,908.001084,908.001084,2024-11-13 12:30:44
28,FF01_Order 57 - Strip 0_Area 36_1.0,725,725,1000.0,1000.0,752.280794,752.280794,2024-11-13 12:30:44
49,FF01_Order 64 - Strip 3_Area 37_1.0,555,555,1000.0,1000.0,752.280794,752.280794,2024-11-13 12:30:44
56,FF01_Order 67 - Strip 0_Area 6_1.0,700,700,1000.0,1000.0,908.001084,752.280794,2024-11-13 12:30:44
72,FF01_Order 71 - Strip 1_Area 8_1.0,420,420,500.0,500.0,1000.000000,1000.000000,2024-11-13 12:30:44
80,FF02_Order 4 - Strip 3_Area 0_1.0,525,525,750.0,750.0,908.001084,908.001084,2024-11-13 12:30:44
91,FF02_Order 14 - Strip 3_Area 18_1.0,554,554,1000.0,1000.0,752.280794,752.280794,2024-11-13 12:30:44


In [21]:
#downlink_result

In [22]:
l1 = 'extracted_raw_file_df', 'selected_oppr_conflict_comparision_df', 'KPI_df'
df111 = result_dict['extracted_raw_file_df']#.columns
#interpret_result_dict.keys()


KeyError: 'extracted_raw_file_df'

In [ ]:
Y = img_capture_result[img_capture_result['operation']=='Imaging']#['StripID']#.nunique()
Z = img_capture_result[img_capture_result['operation']=='downlinking_from_Readout']
Z['duration'] = Z['end_time']- Z['start_time']
#Y[Y['StripID']=='']

#Y[Y['encoded_strip_id']=='Order 1 - Strip 0_Area 0']
Y['StripID'].nunique(),len(Y),len(Z),Z['duration'].sum(),gs_pass_result_df['duration'].sum(),len(gs_pass_result_df)


In [ ]:
Y['SatID'].unique()

In [ ]:
CI = Y[Y['SatID']=='FF03']
RZO = img_capture_result[img_capture_result['operation']=='Readout']
CR = RZO[RZO['SatID']=='FF03']
pd.concat([CI,CR]).sort_values(by='start_time')



In [ ]:
CR

In [ ]:
#'Order 1 - Strip 0_Area 0'
RZO = img_capture_result[img_capture_result['operation']=='Readout']
RZO[RZO['SatID']=='FF01']
RZO



In [ ]:
RZ = capture_plan_data_input['dedicated_readout_df']
len(RZ[RZ['SatID']=='FF01'].sort_values(by='start_time')),len(RZ[RZ['SatID']=='FF02'].sort_values(by='start_time')),len(RZ[RZ['SatID']=='FF03'].sort_values(by='start_time'))
RZ[RZ['SatID']=='FF01'].sort_values(by='start_time')

In [ ]:
img_capture_result['operation'].unique()

[ item for item in capture_plan_data_input['Memory_NoimageGs_TW_list'] if item[2]=='FF01']
#capture_plan_data_input['dedicatedReadoutTWlist__concat_sat_memoryTWindex']['FF01_136.0']

In [ ]:
capture_plan_data_input['active_assured_strip_id_list']

In [ ]:
#filtered_eclipse_data.csv




In [13]:
#eclipse_df[eclipse_df['eclipse']==0]

In [ ]:
eclipse_df.sort_values(by='time_index')

In [ ]:
#config

In [ ]:
#SatID,start_time,end_time,eclipse

In [ ]:
#['GS_pass_df', 'image_opportunity_df', 'image_downlink_df', 'eclipse_df_dict', 'config']
#input_dict['eclipse_df_dict']['FF02']

In [ ]:
capture_plan_data_input['readout_memory_capacity__s']

In [ ]:
capture_plan_data_input['imgery_sat_id_list']#.keys()
[s+'_'+str(n) for s in capture_plan_data_input['imgery_sat_id_list']+capture_plan_data_input['only_gs_sat_id_list'] \
 if s in capture_plan_data_input['dedicatedReadoutTWIndex__sat'].keys() for n in capture_plan_data_input['dedicatedReadoutTWIndex__sat'][s]]



In [ ]:
#img_capture_result[img_capture_result['operation']=='Imaging']

#eclipse_df_dict

In [ ]:
capture_plan_data_input['active_assured_strip_id_list']

In [ ]:
capture_plan_data_input['assured_tasking_based_on_input_list']

In [ ]:
capture_plan_data_input['cs1j2k2Domainlist__cs1j1k1']['FF01_Order 1 - Strip 0_Area 0_1.0']

In [9]:
capture_plan_data_input['GS_Pass_time_objective'] = 0

In [ ]:
#len(img_capture_result[img_capture_result['operation']=='Imaging'])

In [ ]:
capture_plan_data_input

## Run till here

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import itertools

fig = go.Figure()


memory_plot_df = result_dict['combined_result']

#memory_plot_df.columns
#memory_plot_df[['SatID','camera_memory_value_endofTW','delta_camera_memory_value_in_this_TW','base_time','OpportunityStartOffset','OpportunityEndOffset#']]
memory_plot_df = memory_plot_df[['SatID','start_time','end_time','operation','camera_memory_value_endofTW','delta_camera_memory_value_in_this_TW','base_time']]
memory_plot_df.sort_values(by=['SatID','start_time'],inplace=True)

memory_plot_df['till_now_max'] = memory_plot_df.groupby('SatID')['end_time'].cummax()
memory_plot_df['prev_max'] = memory_plot_df.groupby('SatID')['till_now_max'].shift(1)

memory_plot_df1 = memory_plot_df[memory_plot_df['start_time'] > memory_plot_df['prev_max'] + 1] 
memory_plot_df1['start_time1'] = memory_plot_df1['prev_max'] + 1 #TODO1 +1 is okay ?
memory_plot_df1['end_time1'] = memory_plot_df1['start_time'] - 1
memory_plot_df1['operation'] = 'idle'

memory_plot_df1 = memory_plot_df1[['SatID','start_time1','end_time1','operation','base_time']]

#memory_plot_df1 = memory_plot_df1.drop(['start_time', 'end_time','till_now_max','prev_max'], axis=1)
memory_plot_df1.rename(columns={'start_time1':'start_time','end_time1':'end_time'},inplace=True)
#imgGS_union_df1 ==> contains TW without img and without gs pass  table without eclipse divide
final_memory_plot_df = pd.concat([memory_plot_df,memory_plot_df1])
final_memory_plot_df.sort_values(by=['SatID','start_time'],inplace=True)

final_memory_plot_df['camera_memory_value_endofTW'] = final_memory_plot_df['camera_memory_value_endofTW'].ffill()
final_memory_plot_df['delta_camera_memory_value_in_this_TW'] = final_memory_plot_df['delta_camera_memory_value_in_this_TW'].fillna(0)
final_memory_plot_df['start_time'] = final_memory_plot_df[['start_time','base_time']].apply(lambda a: pd.to_datetime(a['base_time']) + pd.DateOffset(seconds=a['start_time']),axis=1)
final_memory_plot_df['end_time'] = final_memory_plot_df[['end_time','base_time']].apply(lambda a: pd.to_datetime(a['base_time']) + pd.DateOffset(seconds=a['end_time']),axis=1)

final_memory_plot_df = final_memory_plot_df[['SatID','start_time','end_time','camera_memory_value_endofTW','delta_camera_memory_value_in_this_TW','operation','base_time']]
sat_list = final_memory_plot_df['SatID'].unique()
#sat_list = ['FF01']


colors = {
        "Imaging": "blue",
        "downlinking_from_Readout": "green",
        "Readout": "red",
        "idle": "gray"
    }


for s in sat_list:
    this_plot_df = final_memory_plot_df[final_memory_plot_df['SatID']==s]
    start_time_list = this_plot_df['start_time'].to_list()
    end_time_list = this_plot_df['end_time'].to_list()
    operation_list = this_plot_df['operation'].to_list()

    camera_memory_value_endofTW_list = this_plot_df['camera_memory_value_endofTW'].to_list()
    this_plot_df['camera_memory_value_startofTW_list'] = this_plot_df['camera_memory_value_endofTW'].shift(1)

    first_row_list = this_plot_df.values.tolist()[0]
    first_row_operation = first_row_list[5]
    first_row_memory_val = first_row_list[3]
    ortherwise_memory_val =  first_row_list[3] - first_row_list[4]
    if first_row_operation not in['Imaging','Readout']:
        this_plot_df['camera_memory_value_startofTW_list'] = this_plot_df['camera_memory_value_startofTW_list'].fillna(first_row_memory_val)
    else:
        this_plot_df['camera_memory_value_startofTW_list'] = this_plot_df['camera_memory_value_startofTW_list'].fillna(ortherwise_memory_val)

    camera_memory_value_StartofTW_list = this_plot_df['camera_memory_value_startofTW_list'].to_list()
    
    time_list = list(itertools.chain.from_iterable(zip(start_time_list,end_time_list )))
    operation_list = list(itertools.chain.from_iterable(zip(operation_list,operation_list )))
    camera_memory_value_list = list(itertools.chain.from_iterable(zip(camera_memory_value_StartofTW_list,camera_memory_value_endofTW_list )))
    df = pd.DataFrame({'time':time_list,'operation':operation_list,'memory':camera_memory_value_list})

    this_list = this_plot_df.values.tolist()
    # Plot using Plotly Express
    # fig = px.line(this_plot_df, x='start_time', y='camera_memory_value_endofTW', color='operation', line_group='operation',
    #             title="Memory Usage of Different Operations")
    
    # fig.add_trace(go.Scatter(
    #                         x=time_list,
    #                         y=camera_memory_value_list,
    #                         mode='lines+markers+text',
    #                         #name=operation_list,
    #                         #line=dict(color=colors.get(operation, 'black'), width=2),
    #                         #text=[f"({start_time}, {current_memory:.2f})", f"({end_time}, {end_memory:.2f})"],
    #                         #textposition=text_position
    #                     ))
    
    # fig.show()
    
    for item in this_list:
            start_time = item[1]
            end_time = item[2]
            current_memory = item[6]
            end_memory = item[3]
            operation = item[5]

            fig.add_trace(go.Scatter(
                            x=[start_time, end_time],
                            y=[current_memory, end_memory],
                            mode='lines+markers+text',
                            name=operation+'_'+s,
                            line=dict(color=colors.get(operation, 'black'), width=2),
                            #text=[f"({start_time}, {current_memory:.2f})", f"({end_time}, {end_memory:.2f})"],
                            #textposition=text_position
                        ))

    #     # Update layout
    fig.update_layout(
        title=f"Memory Profile Over Time",
        xaxis_title="Time",
        yaxis_title="Memory",
        legend_title="Operations",
          )
    # names = set()
    # fig.for_each_trace(
    #     lambda trace:
    #         trace.update(showlegend=False)
    #         if (trace.name in names) else names.add(trace.name))

    # Save figure as HTML
    html_filename = "memory_profile_.html"
    fig.write_html('5_output_data'+'/'+html_filename)

# final_memory_plot_df[final_memory_plot_df['SatID']=='FF01'].sort_values(by='start_time')

import pygwalker as pyg
pyg.walk(final_memory_plot_df)






In [ ]:
help(pyg)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import json
import math

from datetime import datetime as dt

import os
from preprocess_1.preprocess_GSpassSelecion import GSPassPreprocess
from model_3.MILP_GSpassSelection_v3 import GSpassSelection
from postprocess_4.postprocess_GSpassSelection import GSpasspostprocess

from preprocess_1.preprocess_imageAqusuition_test import ImageAquisitionProcess #preprocess_imageAqusuition_test,preprocess_imageAquisition_v3_18112024
from model_3.MILP_imageCapture_v3_17112024 import ImageCapturePlan # MILP_imageCapture_v2_16102024,MILP_imageCapture_v2_25102024 # MILP_imageCapture_v2_07112024 #MILP_imageCapture_v3_17112024
from postprocess_4.image_capture_postprocess_V3_17112024 import ImagecapturePostProcess # image_capture_postprocess_V3_17112024# image_capture_postprocess_v2_18102024

from preprocess_1.preprocess_downlink_WIP import DownlinkingPreProcess
from model_3.MILP_downlink import ImageDownlinkPlan
from postprocess_4.postprocess_downlink import ImageDownlinkPostProcess

from result_interpret import interpret_result
from utils import *

# script_dir = os.path.abspath( os.path.dirname( __file__ ) )
# print("script directory: ",script_dir)
'''
readout is happening after the last image and before entering the eclipse region.
Any heating operation will start if the temp reaches around intial tempertaure.
Higher number of Global priority is assumed to be Higher prior Important image.
if end date of due date is less than 24 hrs from the reference time offset then it is going to assured tasking.
Need offset from when scheduling is started. Example if scheduling is  needed from 10:00AM to 11 PM . Then offset is needed from 10:00 AM or 09:59 AM. Due date based assured tasking is the reason.
Iniial Camera Memory is needed at the start of the any oppr imaging/gsPass whichever is first. 
Iniial Readout Memory is needed at the start of the readout oppr.
Initial power is needed at start of the opportunity(imaging/gspass).For Now (since power constraint is not there for readout)
Iniial thermal value is needed before the start of the oppr imaging/gsPass/readout according to device (for NCCms:readout , for camera detector: Imaging ,For XBT : gs Pass oppr).
Eclipse Event should be starting from first oppr either gsPass/Imaging. Readout is happening after the first imaging so i guess not needed at readout.
'''
'''
Need offset from when scheduling is started. Example if scheduling is  needed from 10:00AM to 11 PM . Then offset is needed from 10:00 AM or 09:59 AM.
Iniial Memory,power and thermal value is needed at 10:00 AM
'''
def select_gs_pass_oppr(GS_pass_df,config):

    obj_preprocess = GSPassPreprocess(GS_pass_df)
    data = obj_preprocess.preprocess()

    obj_model = GSpassSelection(data,config)
    result,thermal_profile_gsPass = GSpasspostprocess(obj_model,data,config).get_gsPasses()# 21 seconds

    try :
        result['duration'] = result['end_time'] - result['start_time']
        result = result[result['duration']> 0]
    except:
        print("model is not converged or infeasible or not solved")

    return result
    

def select_img_opprtunity(image_opportunity_df,gs_pass_result_df,eclipse_df_dict,config):

    #basic flters
    #image_opportunity_df = image_opportunity_df[image_opportunity_df['OpportunityEndOffset']<config['scheduled_Hrs']*3600]
    image_opportunity_df = image_opportunity_df[image_opportunity_df['CloudCoverLimit']>image_opportunity_df['CloudCover']]
    image_opportunity_df = image_opportunity_df[image_opportunity_df['OffNadirLimit']>image_opportunity_df['OffNadir']]

    obj_preprocess = ImageAquisitionProcess(image_opportunity_df,gs_pass_result_df,eclipse_df_dict,config)
    data = obj_preprocess.preprocess()
    #print(data['cs1j2k2Domainlist__cs1j1k1'])

    #++++++++++++++++++++++++++  STEP 0  +++++++++++++++++++++++++++++++++++++++++++++++
    '''
    hard code some data
    '''
    data['camera_memory_capacity__s'] = {s:v for s,v in data['camera_memory_capacity__s'].items() }
    data['readout_memory_capacity__s'] = {s:v for s,v in data['readout_memory_capacity__s'].items() }
    data['power_capacity__s']  = {s:720000000 for s,v in data['power_capacity__s'].items() }
    data['initial_power_value__s']  = {s:v*0.3 for s,v in data['power_capacity__s'].items() }
    #++++++++++++++++++++++++++  STEP 1  +++++++++++++++++++++++++++++++++++++++++++++++
    config['objective']['GS_Pass_time'] = True
    config['objective']['total_priority'] = False
    config['objective']['total_readout_memory'] = False
    obj_model = ImageCapturePlan(data,config)

    #Readout Schedule 
    # data['GS_Pass_time_objective'] = obj_model.prob.objective.value()
    # config['objective']['GS_Pass_time'] = False
    # config['objective']['total_priority'] = False
    # config['objective']['total_readout_memory'] = True
    # obj_model = ImageCapturePlan(data,config)

    #++++++++++++++++++++++++++  STEP 2  +++++++++++++++++++++++++++++++++++++++++++++++

    # config['objective']['GS_Pass_time'] = False
    # config['objective']['total_priority'] = True
    # #config['objective']['total_readout_memory'] = False

    # obj_model = ImageCapturePlan(data,config)

    # data['total_priority_objective'] = obj_model.prob.objective.value()
    #++++++++++++++++++++++++++  STEP 3  +++++++++++++++++++++++++++++++++++++++++++++++

    # config['objective']['total_priority'] = False
    # config['objective']['total_readout_memory'] = True

    # obj_model = ImageCapturePlan(data,config)
    #++++++++++++++++++++++++++  PostProcess  +++++++++++++++++++++++++++++++++++++++++++++++
    post_obj = ImagecapturePostProcess(obj_model,data)
    img_capture_result= post_obj.get_schedule()
    #.isnull().sum()
    return img_capture_result,data
    

    #======================================================================================================================================================================================================

    pass

def get_aps_success_metric(img_capture_result,data):
    after_aps_plan_df = img_capture_result[img_capture_result['operation']=='Imaging']
    criteria_list = ['total_opprtunities_ratio','GP_ratio','LP_ratio','conflictImg_gsPass']

    total_conflict_images_list= [data['success_metric_before']['conflict_images']]
    GP_before = data['success_metric_before']['original_Total_GP']
    LP_before = data['success_metric_before']['original_Total_LP']
    TOppr_before = data['success_metric_before']['total_opportunities'] 
    before_list = [TOppr_before,GP_before,LP_before,total_conflict_images_list]

    GP_after= after_aps_plan_df['mean_global_priority'].sum()
    LP_after = after_aps_plan_df['mean_local_priority'].sum()
    TOppr_after= after_aps_plan_df['encoded_strip_id'].nunique()
    fraction_conflict_images_list = [list(after_aps_plan_df[after_aps_plan_df['encoded_strip_id'].isin(total_conflict_images_list)]['encoded_strip_id'].unique())]
    after_list = [TOppr_after,GP_after,LP_after,fraction_conflict_images_list]

    APS_success_metric_df = pd.DataFrame({'criteria':criteria_list,'potential_input':before_list,'APS_selected':after_list})
    #APS_success_metric_df['percentage'] = APS_success_metric_df['APS_selected'] / APS_success_metric_df['potential_input']
    APS_success_metric_df1 = APS_success_metric_df[:-1]
    APS_success_metric_df1['percentage'] = APS_success_metric_df1['APS_selected'] / APS_success_metric_df1['potential_input'] * 100
    APS_success_metric_df2 = APS_success_metric_df[-1:]

    APS_success_metric_df = pd.concat([APS_success_metric_df1,APS_success_metric_df2])

    return APS_success_metric_df
    

def get_downlink_schedule(image_downlink_df,img_capture_result,config):

    downlink_operation_list  = ['downlinking_from_camera','downlinking_from_Readout']
    img_capture_result_downlink = img_capture_result[img_capture_result['operation'].isin(downlink_operation_list)]
    DownlinkingPreProcessObj = DownlinkingPreProcess(image_downlink_df,img_capture_result_downlink,config)
    data_downlink = DownlinkingPreProcessObj.preprocess()
    
    if config['downlink_schedule_OnlyJustsortImgID']:
        downlink_result = pd.DataFrame(data_downlink['LP_DD_Priority_imgID'].items(),columns=['ImageID','computed_priority']).\
            sort_values(by='computed_priority',ascending=False)
    else:
        obj_downlink_model = ImageDownlinkPlan(data_downlink,config)
        downlink_result = ImageDownlinkPostProcess(obj_downlink_model,data_downlink).get_schedule()
        downlink_result = downlink_result[downlink_result['TileStripNo_downLoad']!=0]

    return downlink_result
    
def schedule():

    pass

def get_input_files(config):
    # GS PASS
    #GS_pass_df = pd.read_csv(config["csv_file_path"]["gs_pass_opportunity"])#APS_gsPasses_TV1#GS_Passes_mock1#GS_Passes_live1#GS_Passes_new (1)
    
    GS_pass_df = pd.read_csv("1_input_data/GS_Passes_new (1).csv")
    GS_pass_df['SatID'] = GS_pass_df['SatID'].astype(str)
    GS_pass_df['AOSOffset'] = GS_pass_df['AOSOffset'].astype(int)
    GS_pass_df['LOSOffset'] = GS_pass_df['LOSOffset'].astype(int)

    # image Opprtunity
    image_opportunity_df = pd.read_csv(config["csv_file_path"]["image_capture_opportunity"])#Imaging_mock1#APS_imagingOpportunities_TV1#Imaging_live#Imaging_new (1)
    # change made priority ulta
    #image_opportunity_df['Priority'] = 1/image_opportunity_df['Priority']
    
    image_opportunity_df['SatID'] = image_opportunity_df['SatID'].astype(str)
    image_opportunity_df['OpportunityStartOffset'] = image_opportunity_df['OpportunityStartOffset'].astype(int)
    image_opportunity_df['OpportunityEndOffset'] = image_opportunity_df['OpportunityEndOffset'].astype(int)
    image_opportunity_df_copy = image_opportunity_df.copy()
    image_opportunity_df_copy['X'] = image_opportunity_df_copy[['OpportunityStartTime','OpportunityStartOffset']].apply(lambda a: pd.to_datetime(a['OpportunityStartTime']) - pd.DateOffset(seconds=a['OpportunityStartOffset']),axis=1)
    image_opportunity_df_copy['Y'] = image_opportunity_df_copy[['OpportunityEndTime','OpportunityEndOffset']].apply(lambda a: pd.to_datetime(a['OpportunityEndTime']) - pd.DateOffset(seconds=a['OpportunityEndOffset']),axis=1)
    base_time_stamp = image_opportunity_df_copy["X"].to_list()[0]
    config['base_time_stamp_downlink'] = base_time_stamp

    #image Downlink
    image_downlink_df = pd.read_csv(config["csv_file_path"]["image_downlink_file"])
    image_downlink_df['assured_downlink_flag'] = [0,0] +[0]*(len(image_downlink_df)-2)
    image_downlink_df['delivery_type'] = 'standard_delivery' # expedited_delivery,super_expedited_delivery
    union_list_of_sat = list(set(image_opportunity_df['SatID']).union(set(GS_pass_df['SatID'])).union(set(image_downlink_df['SatID'])))
    hrs = config['scheduled_Hrs']

    # get dummy eclipse data close to reality
    #eclipse_df = pd.DataFrame()
    #for sat in satellite_list:
        #this_eclipse_df = eclipse_event_df[eclipse_event_df['SatID']==sat]
        #that_eclipse_df = get_eclipse_data(this_eclipse_df,config)
        #eclipse_df = pd.concat([that_eclipse_df,eclipse_df])
        
    min_time_index= min([image_opportunity_df['OpportunityStartOffset'].min(),image_opportunity_df['OpportunityEndOffset'].max(),GS_pass_df['AOSOffset'].min(),GS_pass_df['LOSOffset'].max()])
    max_time_index= max([image_opportunity_df['OpportunityStartOffset'].min(),image_opportunity_df['OpportunityEndOffset'].max(),GS_pass_df['AOSOffset'].min(),GS_pass_df['LOSOffset'].max()])

    hrs = (max_time_index - min_time_index)/3600
    hrs = math.ceil(hrs)
    while True:
        hrs += 1
        if hrs % 1.5==0:
            break


    in_orbit_eclipse_event = [1 for i in range(int(1.5*3600*0.6))] + [0 for i in range(int(1.5*3600*0.4))] #
    eclipse_df  = pd.DataFrame({'time_index': [i for i in range(min_time_index,min_time_index+hrs*3600)] ,"eclipse" : in_orbit_eclipse_event*int(hrs/1.5)})
    eclipse_df['SatID']= [union_list_of_sat] *len(eclipse_df)
    eclipse_df = eclipse_df.explode('SatID')
    eclipse_df_dict = {s: eclipse_df[eclipse_df['SatID']==s] for s in eclipse_df['SatID'].unique()}

    #eclipse_df = pd.DataFrame()
    #for sat in satellite_list:
        #this_eclipse_df = eclipse_event_df[eclipse_event_df['SatID']==sat]
        #that_eclipse_df = get_eclipse_data(this_eclipse_df,config)
        #eclipse_df = pd.concat([that_eclipse_df,eclipse_df])
   #eclipse_df_dict = {s: eclipse_df[eclipse_df['SatID']==s] for s in eclipse_df['SatID'].unique()}

    
    # get dummy data for assured tasking
    image_opportunity_df['encoded_stripId'] =   image_opportunity_df['StripID'].astype(str)+ '_' + image_opportunity_df['AoiID'].astype(str)
    total_capture_list = list(image_opportunity_df['encoded_stripId'].unique())
    no_of_list = len(total_capture_list)
    assured_capture_df = pd.DataFrame({'encoded_stripId':total_capture_list,'assured_task':[0,0]+[0]*(no_of_list-2)})
    image_opportunity_df = pd.merge(image_opportunity_df,assured_capture_df,on='encoded_stripId',how='left')
    image_opportunity_df = image_opportunity_df.drop(columns=['encoded_stripId'])

    # further processing eclipse data to align with gs pass where entire gs pass is assumed to be in eclipse region
    gsPassInput_df_copy = pd.read_csv(config["csv_file_path"]["gs_pass_opportunity"])
    gsPassInput_df_copy['SatID'] = gsPassInput_df_copy['SatID'].astype(str)
    gsPassInput_df_copy['AOSOffset'] = gsPassInput_df_copy['AOSOffset'].astype(int)
    gsPassInput_df_copy['LOSOffset'] = gsPassInput_df_copy['LOSOffset'].astype(int)
    gsPassInput_df_copy['list'] =  gsPassInput_df_copy[['AOSOffset','LOSOffset']].apply(lambda a : [i for i in range(a['AOSOffset'],a['LOSOffset']+1)],axis =1 )

    gsPassInput_df_copy1 = gsPassInput_df_copy[['SatID','list']]
    gsPassInput_df_copy1 = gsPassInput_df_copy1.explode('list')
    gsPassInput_df_grouped_copy1 = gsPassInput_df_copy1.groupby('SatID').agg(time_index_list = ('list',list)).reset_index()
    gsPasstimeIndexList__s = dict(zip(gsPassInput_df_grouped_copy1['SatID'],gsPassInput_df_grouped_copy1['time_index_list']))
    for k,v in eclipse_df_dict.items():
        if k in gsPasstimeIndexList__s.keys():
            this_time_index_list = gsPasstimeIndexList__s[k]
            v.loc[v["time_index"].isin(this_time_index_list), "eclipse"] = 1
            eclipse_df_dict[k] = v

    return {
            'GS_pass_df':GS_pass_df,\
            'image_opportunity_df':image_opportunity_df,\
            'image_downlink_df':image_downlink_df,\
            "eclipse_df_dict": eclipse_df_dict,
            "config":config
            }
    
if __name__ == "__main__":

    # Open and read the JSON file
    with open('1_input_data/config.json', 'r') as file:
        config = json.load(file)

    # if memory constraint False then thermal_constraint is also False
        config['constraints']['thermal_constraint_readout'] = config['constraints']['memory_constrant'] and config['constraints']['thermal_constraint_readout']
        config['constraints']['thermal_constraint_imaging'] = config['constraints']['memory_constrant'] and config['constraints']['thermal_constraint_imaging']

    #======================================================================================================================================================================================================
    # read_input
    input_dict = get_input_files(config)
    config = input_dict['config']
    #======================================================================================================================================================================================================
    #gs pass_selection
    gs_pass_result_df = select_gs_pass_oppr(input_dict['GS_pass_df'],config)
    gs_pass_result_df['Eclipse'] = 1 ## dummy
    gs_pass_result_df['duration'] = gs_pass_result_df['end_time'] - gs_pass_result_df['start_time']
    gs_pass_result_df = gs_pass_result_df[gs_pass_result_df['duration']> 0]
    interpret_gs_pass_result_df_copy = gs_pass_result_df.copy()# this not the gsPass result as it is to be get filtered after due to other factors in image capture plan.It is just to get require info in interpret result.

    print("image_capture_plan_starting")
    #======================================================================================================================================================================================================
    #image_selection
    img_capture_result,capture_plan_data_input= select_img_opprtunity(input_dict['image_opportunity_df'],gs_pass_result_df,input_dict['eclipse_df_dict'],config)
    img_capture_result['base_time'] = config['base_time_stamp_downlink']
    #img_capture_result = img_capture_result[img_capture_result['operation']=='Imaging']
    #readout_result = img_capture_result[img_capture_result['operation']=='Readout']
    interpret_img_capture_resul_copy = img_capture_result.copy()
    #======================================================================================================================================================================================================
    # get APS success metrics 
    APS_success_metric_df = get_aps_success_metric(img_capture_result,capture_plan_data_input)
    #======================================================================================================================================================================================================
    print("Downlink_plan_starting")
    try:
        print(dfd)
        downlink_result = get_downlink_schedule(input_dict['image_downlink_df'],img_capture_result,config)
    except:
        print("downlink_schedule_has_some_error")
        downlink_result = pd.DataFrame()
    downlink_result['base_time'] = config['base_time_stamp_downlink']
    #======================================================================================================================================================================================================
    #img_capture_result[img_capture_result['download_from_']]
    gs_pass_result_df.to_csv("5_output_data/gs_pass_result_df.csv",index=None)
    img_capture_result.to_csv("5_output_data/img_capture_schedule.csv",index=None)
    APS_success_metric_df.to_csv("5_output_data/APS_success_metric.csv",index = None)
    downlink_result.to_csv("5_output_data/downlink_result.csv",index = None)

    interpret_image_opportunity_df = pd.read_csv(config["csv_file_path"]["image_capture_opportunity"])
    interpret_result_dict = interpret_result(interpret_image_opportunity_df,interpret_gs_pass_result_df_copy,interpret_img_capture_resul_copy,config)
    for k,v in interpret_result_dict.items():
        v['base_time'] = config['base_time_stamp_downlink']
        v.to_csv("5_output_data/"+k+".csv",index = None)

    
    only_img_capture_result = img_capture_result[img_capture_result['operation']=='Imaging'][['SatID','start_time','end_time','AoiID','StripID','base_time']]
    only_img_capture_result['start_time'] = only_img_capture_result[['start_time','base_time']].apply(lambda a: pd.to_datetime(a['base_time']) + pd.DateOffset(seconds=a['start_time']),axis=1)
    only_img_capture_result['end_time'] = only_img_capture_result[['end_time','base_time']].apply(lambda a: pd.to_datetime(a['base_time']) + pd.DateOffset(seconds=a['end_time']),axis=1)

    only_readout_result = img_capture_result[img_capture_result['operation']=='Readout'][['SatID','start_time','end_time','base_time']]
    only_readout_result['start_time'] = only_readout_result[['start_time','base_time']].apply(lambda a: pd.to_datetime(a['base_time']) + pd.DateOffset(seconds=a['start_time']),axis=1)
    only_readout_result['end_time'] = only_readout_result[['end_time','base_time']].apply(lambda a: pd.to_datetime(a['base_time']) + pd.DateOffset(seconds=a['end_time']),axis=1)

    result_dict = {"only_readout_result":only_readout_result,\
                  "only_img_capture_result":only_img_capture_result}
    result_dict.update(interpret_result_dict)

    #print(only_img_capture_result,only_readout_result,downlink_result)
#config['constraints'] = ['Thermal_constraints']

In [ ]:
def get_conflicting_dict(self,df,data_dict,different_setup_time,conflicting_on = 'GsID',concat_filter ='concat_gsid_satid_TWIndex',LOS_column='LOSOffset',AOS_column='AOSOffset'):

    for on_item in df[conflicting_on].unique():
        this_df = df[df[conflicting_on] == on_item ]
        # if different_master_key:
        #     data_dict[different_master_key]['sgk_list'] [on_item] = this_df[concat_filter].unique()
        for csgk in this_df[concat_filter].unique():
            that_df = this_df[this_df[concat_filter] == csgk]
            this_LOS = list(that_df[LOS_column].unique())[0]
            this_AOS = list(that_df[AOS_column].unique())[0]


            that_df1 = this_df[this_df[AOS_column] >= different_setup_time  + this_LOS]
            that_df2 = this_df[this_df[LOS_column] <= this_AOS - different_setup_time]
            that_df3 = pd.concat([that_df1,that_df2])
            
            not_needed = list(that_df3[concat_filter].unique())
            that_df = this_df[~this_df[concat_filter].isin(not_needed)]

            that_df = that_df[that_df[concat_filter] != csgk]

            # if different_master_key:
            #     data_dict[different_master_key]['domain_of_csgk'] [csgk] = list(that_df[concat_filter].unique())
            # else:
            data_dict[csgk] = list(that_df[concat_filter].unique())
                
    return data_dict 




In [ ]:
import math
def evaluate_eqn(t,temp_eqn):
    t = t
    return eval(temp_eqn)



from scipy.interpolate import interp1d
import numpy as np
heat_eqn = "0.0000000728690314 * t**3 - 0.000138692964 * t**2 + 0.103057817 * t  + 1.88504399 "
# y = "math.exp(x)"
T = 25
initial_temp = 25
limit_temp = initial_temp + 3


a20 = -13.339128	
b20 = 0.504271	
c20 = 0.581774

a40 = -13.503356	
b40 = 0.655062	
c40 = 0.397629

y20 = 54.9
y40 = 70.6


from scipy.interpolate import interp1d
 
Y = [[-13.339128,-13.503356],[0.504271,0.655062],[0.581774,0.397629],[y20,y40]]
X = [[20,40],[20,40],[20,40],[20,40]]
 
# test value
interpolate_x = initial_temp
 
# Finding the interpolation
l1 =[]
for i in range(4):
    y_interp = interp1d(X[i], Y[i])
    l1.append(y_interp(interpolate_x))

# print("Value of Y at x = {} is".format(interpolate_x),
#       y_interp(interpolate_x))
cool_eqn = str(l1[2])+"*math.exp(1*t*"+str(l1[0])+")+"+str(l1[1])
x = 2
#eval(y)
temp_profile_df = pd.DataFrame()
for i in range(1):
    df = pd.DataFrame({"time_index":[i for i in range(1,T*60)]})
    df['epoch'] = str(i)+'_'+'heat'
    df['delta_temp_heat'] = df['time_index'].apply(lambda a:evaluate_eqn(a,temp_eqn = heat_eqn) ) 
    df['initial_temp'] =  initial_temp
    df['temp']=df['delta_temp_heat']  + df['initial_temp']
    df = df[df['temp']<=70]
    #print(df)
    max_heat_temp = df['temp'].max()
    print(max_heat_temp)
    df_cool = pd.DataFrame({"time_index":[i for i in range(1,T*60)]})
    df_cool['epoch'] = str(i)+'_'+'cool'
    df_cool["time_index_new"] = df_cool["time_index"]/1800
    df_cool['delta_temp'] = df_cool['time_index_new'].apply(lambda a:evaluate_eqn(a,temp_eqn = cool_eqn) ) 
    df_cool['y'] = df_cool['delta_temp']
    df_cool['delta_temp'] = df_cool['delta_temp']* l1[3]
    df_cool['initial_temp'] = max_heat_temp
    df_cool['temp'] = df_cool['initial_temp'] - df_cool['delta_temp'] 

    df_cool['temp'] = df_cool['delta_temp']#* l1[3]
    
    df_cool_check = df_cool[df_cool['temp']<= limit_temp]
    cooled_upto = df_cool_check['temp'].max()
    #df_cool = df_cool[df_cool['temp']>=limit_temp]
    initial_temp = cooled_upto
    
    this_df = pd.concat([df,df_cool])
    temp_profile_df = pd.concat([temp_profile_df,this_df])




#
# df["time_index_new"] = df["time_index"]/60/1800
# df['delta_temp'] = df['time_index_new'].apply(lambda a:evaluate_eqn(a,temp_eqn = cool_eqn) ) 
# #
# xs = np.arange(10)
# ys = 2*xs + 1

# interp_func = interp1d(xs, ys)

# newarr = interp_func(np.arange(2.1, 3, 0.1))

# print(newarr)
# eval(Y, globals, locals)





In [ ]:
cool_eqn

In [ ]:
l1[3]

In [ ]:
df#[df['time_index']<=25*60]
df_cool['delta_temp'].max()
df_cool
temp_profile_df
df_cool
#df#[df['time_index']==1499]

In [ ]:
temp_profile_df[temp_profile_df['epoch']=='1_cool']

In [ ]:
y = "math.exp(x)"
x = 1
eval(y)

In [ ]:
l1